In [3]:
from gzip import READ
from flask import Flask, request, render_template, redirect, url_for, flash, jsonify, send_from_directory, current_app
import requests
from string import Template
from flask_sqlalchemy import SQLAlchemy
from numpy import isnan
import sqlalchemy
import pandas as pd
pd.options.mode.chained_assignment = None
from pandas import DataFrame
from pandas.util import hash_pandas_object
import re
from flask_gtts import gtts
from config import GMAIL_PASSWORD, GMAIL_USERNAME, Config, S3_KEY, S3_SECRET, S3_BUCKET, SES_REGION_NAME, SES_EMAIL_SOURCE, GMAIL_USERNAME, GMAIL_PASSWORD, SERVER_NAME, SECRET_KEY
from flask_login import LoginManager
from models import LoginForm, RegistrationForm, PasswordResetForm, PasswordChangeForm, upload_file_to_s3, time_ago
from wtforms import validators
from wtforms.fields.html5 import EmailField
import email_validator
from passlib.hash import sha256_crypt
from flask_login import login_user, logout_user, login_required, current_user
import bcrypt
from datetime import datetime, timedelta
from dateutil import tz
from colour import Color
import re
import boto3, botocore
from django.utils.crypto import get_random_string
from werkzeug.utils import secure_filename
import os
import base64
import six
import uuid
import imghdr
import io
import pytz
import models


# Not the entire world, just your best friends. 
application = Flask(__name__)
application.secret_key = application.config['SECRET_KEY']

application.config.from_object(Config)


#This is for users table
db = SQLAlchemy(application, engine_options={"pool_recycle": 1800})
db.init_app(application)

##Create SQL Engine Look at this: https://docs.sqlalchemy.org/en/14/core/pooling.html#pool-disconnects
engine = sqlalchemy.create_engine(application.config['SQLALCHEMY_DATABASE_URI'], pool_recycle=3600,)

In [4]:

class User(db.Model):

    __tablename__ = 'users'
    email = db.Column(db.String, primary_key=True)
    id = db.Column(db.String)
    handle = db.Column(db.String)
    password = db.Column(db.String)
    authenticated = db.Column(db.Boolean, default=False)
    
    def is_active(self):
        """True, as all users are active."""
        return True
    
    def get_id(self):
        """Return the email address to satisfy Flask-Login's requirements."""
        return self.email

    def get_user_id(self):
        """Return the email address to satisfy Flask-Login's requirements."""
        return self.id   

    def get_user_handle(self):
        """Return the email address to satisfy Flask-Login's requirements."""
        return self.handle         
        
    def is_authenticated(self):
        """Return True if the user is authenticate#d."""
        return self.authenticated


In [8]:
def get_president_user():
    with engine.connect() as connection:
        ResultProxy = connection.execute('''SELECT  u.id, u.profile_photo, SUM(p1.value) AS user_score
                                                FROM users u
                                                LEFT JOIN posts p ON p.user_id = u.id
                                                LEFT JOIN post_votes p1 ON p1.post_id = p.post_id
                                                GROUP BY u.id
                                                ORDER BY user_score DESC
                                                LIMIT 1;''')
        df = DataFrame(ResultProxy.fetchall())
        df.columns = ResultProxy.keys()
        president_user_id = df['id'][0]
        return president_user_id

def is_president(list_of_user_ids):
    president_user_id = get_president_user()
    #retun list of true or false if users are president
    is_president = []
    for user_id in list_of_user_ids:
        if user_id == president_user_id:
            is_president.append(True)
        else:
            is_president.append(False)
    return is_president

In [23]:
def get_user_posts(current_user_id, search_user_id, last_post_id):

    #get max post_id if last_post_id is None
    if last_post_id is None:
        with engine.connect() as connection:
            result = connection.execute("""SELECT MAX(post_id) FROM posts
                                         WHERE user_id = %s;""", search_user_id)

            last_post_id = result.fetchone()[0] + 1

    with engine.connect() as connection:
        ResultProxy = connection.execute("""SELECT p.post_id, p.user_id, u.first_name, u.handle, u.profile_photo, p.reply_to_id, p.creation_time, pv.post_score, p.post_text, b.user_score, COALESCE(c.current_user_vote, 0 ) as current_user_vote 
                                                FROM posts p
                                                LEFT JOIN users u ON u.id = p.user_id 
                                                LEFT JOIN 
                                                    (
                                                        SELECT f.user_id, f.following, f.follow_value
                                                            FROM follows f
                                                            WHERE f.user_id = %s AND f.follow_value = 1
                                                    ) f ON f.following = p.user_id 
                                                LEFT JOIN
                                                    (
                                                        SELECT pv.post_id, SUM(pv.value) AS post_score
                                                            FROM post_votes pv
                                                            GROUP BY pv.post_id
                                                    ) pv ON p.post_id = pv.post_id
                                                LEFT JOIN
                                                        (
                                                            SELECT u.id, SUM(p1.value) AS user_score
                                                                FROM users u
                                                                LEFT JOIN posts p ON p.user_id = u.id
                                                                LEFT JOIN post_votes p1 ON p1.post_id = p.post_id
                                                                GROUP BY u.id
                                                        ) b ON b.id = u.id
                                                LEFT JOIN
                                                        (
                                                        SELECT p2.post_id, SUM(p2.value) AS current_user_vote
                                                            FROM post_votes p2
                                                            WHERE p2.user_id = %s
                                                            GROUP BY p2.post_id
                                                        ) c on c.post_id = p.post_id 
                                            WHERE p.post_id < %s
                                            AND p.user_id = %s
                                            AND p.reply_to_id IS NULL AND p.is_deleted = 0
                                            ORDER BY p.post_id DESC
                                            LIMIT 10; """, (current_user_id, current_user_id, last_post_id, search_user_id))
        df = DataFrame(ResultProxy.fetchall())
    

    if len(df.index) > 0:
        df.columns = ResultProxy.keys()

        #Get comments and scores for each post_id
        ids = ', '.join(f'{w}' for w in df.post_id)
        ids = "(" + ids + ")"

        with engine.connect() as connection:
            ResultProxy = connection.execute("""SELECT p.post_id, p2.reply_count, pv.down_votes, pv2.up_votes
                                                        FROM posts p
                                                        LEFT JOIN
                                                            (
                                                                SELECT p.reply_to_id, COUNT(p.post_id) AS reply_count
                                                                    FROM posts p
                                                                    WHERE p.reply_to_id IN %s AND p.is_deleted = 0
                                                                    GROUP BY p.reply_to_id
                                                            ) p2 ON p2.reply_to_id = p.post_id
                                                        LEFT JOIN
                                                            (
                                                                SELECT pv.post_id, COUNT(pv.value) AS down_votes
                                                                    FROM post_votes pv
                                                                    WHERE pv.post_id IN %s AND pv.value < 0
                                                                    GROUP BY pv.post_id
                                                            ) pv ON pv.post_id = p.post_id
                                                        LEFT JOIN
                                                            (
                                                                SELECT pv.post_id, COUNT(pv.value) AS up_votes
                                                                    FROM post_votes pv
                                                                    WHERE pv.post_id IN %s AND pv.value > 0
                                                                    GROUP BY pv.post_id
                                                            ) pv2 ON pv2.post_id = p.post_id	
                                                        WHERE p.post_id IN %s; """ % (ids, ids, ids, ids))
            
        df2 = DataFrame(ResultProxy.fetchall())
        df2.columns = ResultProxy.keys()
        
        df = pd.merge(df, df2, on=['post_id'], how='left')

        #check if user is president
        df['is_president'] = is_president(df['user_id'])
        
        return df

In [25]:
df = get_user_posts(8, 8, 1184)
df = models.format_feed(df)

   post_id  user_id first_name handle     profile_photo reply_to_id  \
0     1182        8       Dave   dave  048af5bb-c3f.jpg        None   
1     1179        8       Dave   dave  048af5bb-c3f.jpg        None   
2     1163        8       Dave   dave  048af5bb-c3f.jpg        None   
3     1157        8       Dave   dave  048af5bb-c3f.jpg        None   
4     1150        8       Dave   dave  048af5bb-c3f.jpg        None   
5     1146        8       Dave   dave  048af5bb-c3f.jpg        None   
6     1145        8       Dave   dave  048af5bb-c3f.jpg        None   
7     1144        8       Dave   dave  048af5bb-c3f.jpg        None   
8     1140        8       Dave   dave  048af5bb-c3f.jpg        None   
9     1139        8       Dave   dave  048af5bb-c3f.jpg        None   

        creation_time post_score  \
0 2022-08-18 15:27:02       None   
1 2022-08-17 03:11:57      1.000   
2 2022-08-15 02:40:56      3.000   
3 2022-08-13 18:02:58      1.000   
4 2022-08-08 17:09:42       None   
5 